In [1]:
from cloudsafe import *

In [2]:
database = 'here-api/main.db'
conn = create_connection(database)

In [3]:
departures = select_all_departures(conn)

In [4]:
stations = select_all_stations(conn)

In [5]:
departures

,id,station_id,bus,headsign,day,time
0,1,1,12,Islev Viemosevej,saturday,00:53
1,2,1,12,Islev Viemosevej,saturday,06:53
2,3,1,12,Islev Viemosevej,saturday,07:53
3,4,1,12,Islev Viemosevej,saturday,08:23
4,5,1,12,Islev Viemosevej,saturday,08:53
...,...,...,...,...,...,...
284966,284967,1047,22,Glostrup St. Glosemosevej,weekday,21:33
284967,284968,1047,22,Glostrup St. Glosemosevej,weekday,22:05
284968,284969,1047,22,Glostrup St. Glosemosevej,weekday,22:26
284969,284970,1047,22,Glostrup St. Glosemosevej,weekday,23:05


In [6]:
stations

,id,id_overpass,name_overpass,coordinates_overpass,id_here,name_here,coordinates_here,no_data,duplicate
0,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0.0,0.0
1,2,node/8084956,Grøndal Station,"55.690687,12.516191",205078064,Grøndal St. (Godthåbsvej),"55.690604,12.516298",0.0,0.0
2,3,node/12723457,"Svanemøllen St., Sløjfen","55.714979,12.578019",205279366,"Svanemøllen St., Sløjfen (Svanemøllen St.)","55.714996,12.577951",0.0,0.0
3,4,node/283018988,Fuglsang Allé,"55.697821,12.501116",205078056,Fuglsang Allé (Annebergvej),"55.69784,12.500979",0.0,0.0
4,5,node/283018989,Bellahøjvej,"55.695901,12.50379",205078060,Bellahøjvej (Primulavej),"55.695969,12.503837",0.0,0.0
...,...,...,...,...,...,...,...,...,...
1046,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0.0,0.0
1047,1048,node/7178922090,Frederikssundsvej,"55.710319,12.477108",None,None,None,NaN,1.0
1048,1049,node/7891926536,Nørreport St.,"55.684077,12.57255",None,None,None,1.0,NaN
1049,1050,node/7891953095,Ryparken St.,"55.714915,12.558111",None,None,None,NaN,1.0


In [7]:
conn.close()